<!--
tags: ["sequential chats"]
description: |
    Use AutoGen to solve a set of tasks with a sequence of chats.
-->

# Auto Generated Agent Chat: Solving a set of tasks with a sequence of chats

This notebook showcases how to use the new chat interface `autogen.initiate_chats` to solve a set of tasks with a sequence of chats. 


## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

### Example Tasks
Below are three example tasks, with each task being a string of text describing the request. The completion of later tasks requires or benefits from the results of previous tasks.


In [4]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")

llm_config = {"config_list": config_list}

financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
    """Investigate possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

### Solve tasks with a series of chats

The `autogen.initiate_chats` interface can take a list of dictionaries as inputs. Each dictionary preserves the following fields: 
- `sender`: a conversable agent as the sender;
- `recipient`: a conversable agent as the recipient;
- `message`: is a string of text (typically a message containing the task);
- `summary_method`: A string specifying the method to get a summary from the chat. Currently supported choices include `last_msg`, which takes the last message from the chat history as the summary, and `reflection_with_llm`, which uses an LLM call to reflect on the chat history and summarize a takeaway;
- `summary_prompt`: A string specifying how to instruct an LLM-backed agent (either the recipient or the sender in the chat) to reflect on the chat history and derive a summary. If not otherwise specified, a default prompt will be used when `summary_method` is `reflection_with_llm`.
"Summarize the takeaway from the conversation. Do not add any introductory phrases. If the intended request is NOT properly addressed, please point it out."
- `carryover`: A string or a list of string to specify additional context to be used in the chat. With `initiate_chats`, summary from previous chats will be added as carryover. They will be appended after the carryover provided by the user.

In [5]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user_proxy_auto = autogen.UserProxyAgent(
    name="User_Proxy_Auto",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="ALWAYS",  # ask human for input at each step
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


chat_results = autogen.initiate_chats(
    [
        {
            "sender": user_proxy_auto,
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "summary_method": "last_msg",
        },
        {
            "sender": user_proxy_auto,
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "summary_method": "reflection_with_llm",
        },
        {
            "sender": user_proxy,
            "recipient": writer,
            "message": writing_tasks[0],
            "carryover": "I want to include a figure or a table of data in the blogpost.",
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

With the following carryover: 


********************************************************************************
User_Proxy_Auto (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User_Proxy_Auto):

To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), along with their performance over the past month in terms of percentage change, we can use a Python library like `yfinance`. This library allows downloading of historical market data from Yahoo Finance.

Let's proceed in the following steps:


#### Check chat results
The `initiate_chat` method returns a `ChatResult` object, which is a dataclass object storing information about the chat. Currently, it includes the following attributes:

- `chat_history`: a list of chat history.
- `summary`: a string of chat summary. A summary is only available if a summary_method is provided when initiating the chat.
- `cost`: a tuple of (total_cost, total_actual_cost), where total_cost is a dictionary of cost information, and total_actual_cost is a dictionary of information on the actual incurred cost with cache.
- `human_input`: a list of strings of human inputs solicited during the chat. (Note that since we are setting `human_input_mode` to `NEVER` in this notebook, this list is always empty.)

In [7]:
for agent, chat_res in chat_results.items():
    print(f"Conversation summary with {agent.name}:")
    print(chat_res.summary)
    print("Human input in the middle:", chat_res.human_input)
    print("Conversation cost: ", chat_res.cost)
    print("\n\n")

Conversation summary with Financial_assistant:
The Python script has successfully executed and provided the following information:

- The current stock price of NVIDIA Corporation (NVDA) is $721.33.
- The current stock price of Tesla, Inc. (TSLA) is $193.57.
- NVIDIA Corporation (NVDA) has seen a 27.94% increase in its stock price over the past month.
- Tesla, Inc. (TSLA) has experienced an 11.98% decrease in its stock price over the past month.

This gives you the current prices and the performance in percentage terms over the last month for both NVDA and TSLA stocks.


Human input in the middle: []
Conversation cost:  ({'total_cost': 0.08696999999999999, 'gpt-4': {'cost': 0.08696999999999999, 'prompt_tokens': 1597, 'completion_tokens': 651, 'total_tokens': 2248}}, {'total_cost': 0})



Conversation summary with Researcher:
The stock performance of NVIDIA Corporation (NVDA) has seen a 27.94% increase, possibly due to the anticipation of a strong upcoming earnings report for the fiscal